In [1]:
import pandas as pd

# Cargar product_id_apredecir201912.txt
df_ids = pd.read_csv('product_id_apredecir201912.txt')
df_ids.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [2]:
# Cargar sell-in.txt (puede ser un archivo grande, leer solo columnas necesarias)
sellin_cols = ['periodo', 'customer_id', 'product_id', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn']
df_sellin = pd.read_csv('sell-in.txt', sep='\t', usecols=sellin_cols)
df_sellin.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [3]:
# Contar valores únicos de customer_id
df_sellin['customer_id'].nunique()
# Contar valores únicos de product_id
#df_sellin['product_id'].nunique()
# Contar valores únicos de periodo
#df_sellin['periodo'].nunique()


597

In [4]:
# Merge de ambos dataframes por product_id
#df_merged = pd.merge(df_sellin, df_ids, on='product_id', how='inner')
#df_merged.head()

In [5]:
# Si 'periodo' es tipo string o int, conviértelo a datetime para mayor facilidad
df_sellin['periodo'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')
df_sellin= df_sellin.sort_values(['product_id', 'customer_id', 'periodo']).reset_index(drop=True)


Hacer el producto cartesiano de producto-cliente-período y agregar 0s.

In [6]:
# 1. Listas de valores únicos
product_ids = df_sellin['product_id'].unique()
customer_ids = df_sellin['customer_id'].unique()
periodos = pd.date_range(
    start=df_sellin['periodo'].min(), 
    end=df_sellin['periodo'].max(), 
    freq='MS'  # Monthly start frequency
)

# 2. Producto cartesiano
cartesian = pd.MultiIndex.from_product(
    [product_ids, customer_ids, periodos], 
    names=['product_id', 'customer_id', 'periodo']
).to_frame(index=False)

In [7]:
# 3. Filtrar productos activos
periodo_producto = df_sellin.groupby('product_id')['periodo'].agg(['min', 'max']).reset_index()
periodo_producto.columns = ['product_id', 'periodo_min_producto', 'periodo_max_producto']

# 4. Filtrar clientes activos
periodo_customer = df_sellin.groupby('customer_id')['periodo'].agg(['min', 'max']).reset_index()
periodo_customer.columns = ['customer_id', 'periodo_min_customer', 'periodo_max_customer']

In [8]:
# 5. Merge para filtrar combinaciones válidas
cartesian = cartesian.merge(periodo_producto, on='product_id', how='left')
cartesian = cartesian.merge(periodo_customer, on='customer_id', how='left')

# 6. Filtrar combinaciones donde el periodo esté dentro del rango activo
cartesian = cartesian[
    (cartesian['periodo'] >= cartesian['periodo_min_producto']) & 
    (cartesian['periodo'] <= cartesian['periodo_max_producto']) &
    (cartesian['periodo'] >= cartesian['periodo_min_customer']) #&
    #(cartesian['periodo'] <= cartesian['periodo_max_customer'])
].copy()

# 7. Merge con el dataset original
df_final = cartesian.merge(
    df_sellin, 
    on=['product_id', 'customer_id', 'periodo'], 
    how='left'
)

In [9]:
df_final.shape

(17173448, 11)

In [10]:
# 8. Completar las ventas y cualquier otra variable faltante con 0
df_final['tn'] = df_final['tn'].fillna(0)
df_final['plan_precios_cuidados'] = df_final['plan_precios_cuidados'].fillna(0)
df_final['cust_request_qty'] = df_final['cust_request_qty'].fillna(0)
df_final['cust_request_tn'] = df_final['cust_request_tn'].fillna(0)
# Podés extender esto a otras variables que consideres relevantes.

# 9. Revisar el resultado
print(df_final.head())

   product_id  customer_id    periodo periodo_min_producto  \
0       20001        10001 2017-01-01           2017-01-01   
1       20001        10001 2017-02-01           2017-01-01   
2       20001        10001 2017-03-01           2017-01-01   
3       20001        10001 2017-04-01           2017-01-01   
4       20001        10001 2017-05-01           2017-01-01   

  periodo_max_producto periodo_min_customer periodo_max_customer  \
0           2019-12-01           2017-01-01           2019-12-01   
1           2019-12-01           2017-01-01           2019-12-01   
2           2019-12-01           2017-01-01           2019-12-01   
3           2019-12-01           2017-01-01           2019-12-01   
4           2019-12-01           2017-01-01           2019-12-01   

   plan_precios_cuidados  cust_request_qty  cust_request_tn         tn  
0                    0.0              11.0         99.43861   99.43861  
1                    0.0              23.0        198.84365  198.84365  

Creación de variable target: tn_t_plus_2

In [11]:
# Creamos la variable objetivo: tn en t+2 para cada combinación product_id y customer_id
# df_merged['tn_t_plus_2']

# Paso 1: Crear columna con periodo +2 meses
df_final['periodo_target'] = df_final['periodo'] + pd.DateOffset(months=2)

# Paso 2: Crear DataFrame con target
target_df = df_final[['product_id', 'customer_id', 'periodo', 'tn']].copy()
target_df.rename(columns={'periodo': 'periodo_target', 'tn': 'tn_t_plus_2'}, inplace=True)

# Paso 3: Hacer el merge
df_final = df_final.merge(
    target_df,
    on=['product_id', 'customer_id', 'periodo_target'],
    how='left'
)

# Paso 4: Validar
print(df_final[['product_id', 'customer_id', 'periodo', 'tn', 'tn_t_plus_2']].head(20))




    product_id  customer_id    periodo         tn  tn_t_plus_2
0        20001        10001 2017-01-01   99.43861     92.46537
1        20001        10001 2017-02-01  198.84365     13.29728
2        20001        10001 2017-03-01   92.46537    101.00563
3        20001        10001 2017-04-01   13.29728    128.04792
4        20001        10001 2017-05-01  101.00563    101.20711
5        20001        10001 2017-06-01  128.04792     43.33930
6        20001        10001 2017-07-01  101.20711    289.35024
7        20001        10001 2017-08-01   43.33930    222.11389
8        20001        10001 2017-09-01  289.35024    111.54944
9        20001        10001 2017-10-01  222.11389    131.27150
10       20001        10001 2017-11-01  111.54944     49.61857
11       20001        10001 2017-12-01  131.27150     88.44065
12       20001        10001 2018-01-01   49.61857    214.72336
13       20001        10001 2018-02-01   88.44065    132.83419
14       20001        10001 2018-03-01  214.72336    16

In [12]:
# Eliminamos filas donde no hay target (NaN)
#df_model = df_merged.dropna(subset=['tn_t_plus_2'])

In [13]:
# Probar combinaciones de producto - cliente
df_final[(df_final['product_id'] == 20524) & (df_final['customer_id'] == 10125)].head(10)

,product_id,customer_id,periodo,periodo_min_producto,periodo_max_producto,periodo_min_customer,periodo_max_customer,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_target,tn_t_plus_2
8959408,20524,10125,2017-01-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,1.0,0.02271,0.02271,2017-03-01,0.00000
8959409,20524,10125,2017-02-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,0.0,0.00000,0.00000,2017-04-01,0.00757
8959410,20524,10125,2017-03-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,0.0,0.00000,0.00000,2017-05-01,0.00757
8959411,20524,10125,2017-04-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,1.0,0.00757,0.00757,2017-06-01,0.00000
8959412,20524,10125,2017-05-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,1.0,0.00757,0.00757,2017-07-01,0.00000
8959413,20524,10125,2017-06-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,0.0,0.00000,0.00000,2017-08-01,0.00000
8959414,20524,10125,2017-07-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,0.0,0.00000,0.00000,2017-09-01,0.00757
8959415,20524,10125,2017-08-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,0.0,0.00000,0.00000,2017-10-01,0.00757
8959416,20524,10125,2017-09-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,1.0,0.00757,0.00757,2017-11-01,0.00000
8959417,20524,10125,2017-10-01,2017-01-01,2019-12-01,2017-01-01,2019-11-01,0.0,1.0,0.00757,0.00757,2017-12-01,0.00000


Feature engeneering

In [14]:
df_full = df_final.copy()

# Elimino columnas innecesarias
df_full.drop(columns=['periodo_min_producto', 'periodo_max_producto', 'periodo_min_customer', 'periodo_max_customer', 'periodo_target'], inplace=True)
df_full.head()

,product_id,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_t_plus_2
0,20001,10001,2017-01-01,0.0,11.0,99.43861,99.43861,92.46537
1,20001,10001,2017-02-01,0.0,23.0,198.84365,198.84365,13.29728
2,20001,10001,2017-03-01,0.0,33.0,92.46537,92.46537,101.00563
3,20001,10001,2017-04-01,0.0,8.0,13.29728,13.29728,128.04792
4,20001,10001,2017-05-01,0.0,15.0,101.20711,101.00563,101.20711


In [15]:
df_full.shape

(17173448, 8)

1. Lags (valores previos)


In [16]:
for lag in range(1, 37):
    df_full[f'tn_lag_{lag}'] = df_full.groupby(['product_id', 'customer_id'])['tn'].shift(lag)

2. Rolling Mean (media móvil)
media de los últimos 3, 6, 9 y 12 meses:

In [17]:
for window in [3, 6, 9, 12, 15, 18, 21, 24]:  #15, 18, 21, 24
    df_full[f'tn_rollmean_{window}'] = (
        df_full.groupby(['product_id', 'customer_id'])['tn']
        .transform(lambda x: x.rolling(window, min_periods=1).mean())
    )

3. Rolling Sum (suma móvil)
la suma de los últimos 3 meses:

In [18]:
df_full['tn_rollsum_3'] = (
    df_full.groupby(['product_id', 'customer_id'])['tn']
    .transform(lambda x: x.rolling(3, min_periods=1).sum())
)

4. Promedio histórico por producto y cliente
Capturar el comportamiento típico de cada combinación:

In [19]:
# Promedio histórico de tn por product_id y customer_id hasta el periodo actual (excluyendo el actual)
df_full['tn_mean_hist'] = (
    df_full.groupby(['product_id', 'customer_id'])['tn']
    .transform(lambda x: x.expanding().mean().shift(1))
)

5. Lag y rolling para otras variables
Repetir la lógica para otras columnas como cust_request_qty y cust_request_tn:

In [20]:
for col in ['cust_request_qty', 'cust_request_tn']:
    for lag in [1, 2, 3]:
        df_full[f'{col}_lag_{lag}'] = df_full.groupby(['product_id', 'customer_id'])[col].shift(lag)
    
    for window in [3, 6, 12]:
        df_full[f'{col}_rollmean_{window}'] = (
            df_full.groupby(['product_id', 'customer_id'])[col]
            .transform(lambda x: x.rolling(window, min_periods=1).mean())
        )

6. Diferencias respecto a valores pasados

In [21]:
for lag in [1, 2, 3, 6]:
    df_full[f'tn_diff_lag_{lag}'] = (
        df_full.groupby(['product_id', 'customer_id'])['tn']
        .transform(lambda x: x - x.shift(lag))
    )


7. Diferencias respecto a rolling mean

In [22]:
for window in [3, 6, 9, 12]:
    rm_col = f'tn_rollmean_{window}'
    df_full[f'tn_diff_rm_{window}'] = df_full['tn'] - df_full[rm_col]


8. Ratios respecto a rolling mean

In [23]:
for window in [3, 6, 9, 12]:
    rm_col = f'tn_rollmean_{window}'
    df_full[f'tn_ratio_rm_{window}'] = df_full['tn'] / (df_full[rm_col] + 0.1)  # evita división por cero


9. Indicadores binarios

In [24]:
df_full['tn_is_increasing'] = (
    df_full.groupby(['product_id', 'customer_id'])['tn']
    .transform(lambda x: (x > x.shift(1)).astype(int))
)

for window in [3, 6]:
    rm_col = f'tn_rollmean_{window}'
    df_full[f'tn_above_rm_{window}'] = (df_full['tn'] > df_full[rm_col]).astype(int)


10. Volatilidad local (qué tan variable es el valor en una ventana)

In [25]:
for window in [3, 6, 12]:
    df_full[f'tn_volatility_{window}'] = (
        df_full.groupby(['product_id', 'customer_id'])['tn']
        .transform(lambda x: x.rolling(window).std())
    )

11. Ratio contra t-1 o rolling anterior (más versiones)

In [26]:
for lag in [1, 2, 3]:
    df_full[f'tn_ratio_lag_{lag}'] = df_full['tn'] / (df_full.groupby(['product_id', 'customer_id'])['tn'].shift(lag) + 0.1)


12. Contadores de inactividad o explosión (períodos que pasaron sin ventas / períodos con crecimiento fuerte)

In [27]:
df_full['tn_zero'] = (df_full['tn'] == 0).astype(int)

for window in [3, 6, 12]:
    df_full[f'tn_zero_count_{window}'] = (
        df_full.groupby(['product_id', 'customer_id'])['tn_zero']
        .transform(lambda x: x.rolling(window, min_periods=1).sum())
    )



13. Features categóricos de comportamiento (Agrupar por cuartiles)

In [29]:
import numpy as np

df_full['tn_quantile'] = (
    df_full.groupby(['product_id', 'customer_id'])['tn']
    .transform(lambda x: pd.qcut(
        x.rank(method='first') if x.nunique() >= 4 else pd.Series([np.nan]*len(x), index=x.index),
        4,
        labels=False,
        duplicates='drop'
    ))
)


14. Features temporales (mes, estacionalidad)

In [30]:
import numpy as np

df_full['month'] = df_full['periodo'].dt.month
df_full['quarter'] = df_full['periodo'].dt.quarter
df_full['year'] = df_full['periodo'].dt.year
df_full['month_sin'] = np.sin(2 * np.pi * df_full['month'] / 12)
df_full['month_cos'] = np.cos(2 * np.pi * df_full['month'] / 12)


In [31]:
# Probar combinaciones de producto - cliente
#df_full[(df_full['product_id'] == 20524) & (df_final['customer_id'] == 10125)].head(10)
df_full.head()

,product_id,customer_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,tn_t_plus_2,tn_lag_1,tn_lag_2,...,tn_zero,tn_zero_count_3,tn_zero_count_6,tn_zero_count_12,tn_quantile,month,quarter,year,month_sin,month_cos
0,20001,10001,2017-01-01,0.0,11.0,99.43861,99.43861,92.46537,NaN,NaN,...,0,0.0,0.0,0.0,0.0,1,1,2017,0.500000,8.660254e-01
1,20001,10001,2017-02-01,0.0,23.0,198.84365,198.84365,13.29728,99.43861,NaN,...,0,0.0,0.0,0.0,2.0,2,1,2017,0.866025,5.000000e-01
2,20001,10001,2017-03-01,0.0,33.0,92.46537,92.46537,101.00563,198.84365,99.43861,...,0,0.0,0.0,0.0,0.0,3,1,2017,1.000000,6.123234e-17
3,20001,10001,2017-04-01,0.0,8.0,13.29728,13.29728,128.04792,92.46537,198.84365,...,0,0.0,0.0,0.0,0.0,4,2,2017,0.866025,-5.000000e-01
4,20001,10001,2017-05-01,0.0,15.0,101.20711,101.00563,101.20711,13.29728,92.46537,...,0,0.0,0.0,0.0,1.0,5,2,2017,0.500000,-8.660254e-01


In [49]:
# Guardar el DataFrame final
df_full.to_parquet("df_full.parquet")

---